In [6]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging, random, re


logging.basicConfig(level=logging.ERROR)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [7]:
df = pd.read_csv("../files/sentiment_dataset/TrainingDatasets/Train50.csv",sep=';')
df = [ [txt,snt] for txt,snt in zip(df.iloc[:,1],df.iloc[:,3]) ]
random.shuffle(df)

In [8]:
for el in df:
    el[0] = re.sub("\:[\w|)|(]*",'', el[0])
    el[0] = re.sub("https//t.co/\w*",'', el[0])
    el[0] = re.sub("\@\w*",'', el[0])
    el[0] = el[0].strip()
    
# Preparing train data
train_df = pd.DataFrame(df[:45000])
train_df.columns = ["text", "labels"]

# Preparing eval data
eval_df = pd.DataFrame(df[45000:])
eval_df.columns = ["text", "labels"]

In [5]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=3,regression=True)

# Create a ClassificationModel
model = ClassificationModel("bert", 
                            "bert-base-multilingual-cased", 
                            num_labels=1, 
                            use_cuda=True,
                            args=model_args)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/45000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/5625 [00:00<?, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasGemmEx( handle, opa, opb, m, n, k, &falpha, a, CUDA_R_16F, lda, b, CUDA_R_16F, ldb, &fbeta, c, CUDA_R_16F, ldc, CUDA_R_32F, CUBLAS_GEMM_DFALT_TENSOR_OP)`

In [ ]:
lyrics = pd.read_csv("../files/4mula_filtered.csv")
lyrics['sent_label'] = np.full(len(lyrics),-100)
lyrics.head(5)

In [ ]:
labels = []
for i, el in lyrics[:1000].iterrows():
    label, raw_outputs = model.predict([lyrics.iloc[i,2]])
    labels.append(float(label))

lyrics.iloc[:1000,3] = labels
lyrics.to_csv('../files/4mula_sent.csv',index=False)